In [9]:
import pickle
import numpy as np
import pandas as pd

from scipy.stats import skew, kurtosis
from sklearn.linear_model import LinearRegression

In [10]:
# change working directory to base, to make all imports and file paths work
import os
#os.chdir(os.pardir)
os.chdir(r"C:\Users\PC-1\Documents\GitHub\ProjectsPrivate\coin_cell_battery")
print("Current directory: %s" % os.getcwd())


Current directory: C:\Users\PC-1\Documents\GitHub\ProjectsPrivate\coin_cell_battery


In [11]:
import src.constants as cst

In [12]:
# window defaults
window_size = 20
shift = 20
stride = 1
batch_size = 16

In [13]:
preprocessed_pkl = pickle.load(open('./data/interim/processed_data.pkl', "rb"))  # dict

In [14]:
preprocessed_pkl.keys()

dict_keys(['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4', 'b1c5', 'b1c6', 'b1c7', 'b1c9', 'b1c11', 'b1c14', 'b1c15', 'b1c16', 'b1c17', 'b1c18', 'b1c19', 'b1c20', 'b1c21', 'b1c23', 'b1c24', 'b1c25', 'b1c26', 'b1c27', 'b1c28', 'b1c29', 'b1c30', 'b1c31', 'b1c32', 'b1c33', 'b1c34', 'b1c35', 'b1c36', 'b1c37', 'b1c38', 'b1c39', 'b1c40', 'b1c41', 'b1c42', 'b1c43', 'b1c44', 'b1c45', 'b2c0', 'b2c1', 'b2c2', 'b2c3', 'b2c4', 'b2c5', 'b2c6', 'b2c10', 'b2c11', 'b2c12', 'b2c13', 'b2c14', 'b2c17', 'b2c18', 'b2c19', 'b2c20', 'b2c21', 'b2c22', 'b2c23', 'b2c24', 'b2c25', 'b2c26', 'b2c27', 'b2c28', 'b2c29', 'b2c30', 'b2c31', 'b2c32', 'b2c33', 'b2c34', 'b2c35', 'b2c36', 'b2c37', 'b2c38', 'b2c39', 'b2c40', 'b2c41', 'b2c42', 'b2c43', 'b2c44', 'b2c45', 'b2c46', 'b2c47', 'b3c0', 'b3c1', 'b3c3', 'b3c4', 'b3c5', 'b3c6', 'b3c7', 'b3c8', 'b3c9', 'b3c10', 'b3c11', 'b3c12', 'b3c13', 'b3c14', 'b3c15', 'b3c16', 'b3c17', 'b3c18', 'b3c19', 'b3c20', 'b3c21', 'b3c22', 'b3c24', 'b3c25', 'b3c26', 'b3c27', 'b3c28', 'b3c29', 'b3c30

In [15]:
preprocessed_pkl['b1c0']['summary'].keys()

dict_keys(['IR', 'QD', 'Remaining_cycles', 'Discharge_time'])

In [16]:
preprocessed_pkl['b1c0']['cycles'].keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157'

In [17]:
preprocessed_pkl['b1c0']['cycles']['1'].keys()

dict_keys(['Qdlin', 'Tdlin', 'Vdlin'])

In [18]:
# fullmod_features = [
#     "minimum_dQ_100_10",
#     "variance_dQ_100_10",
#     "slope_lin_fit_2_100",
#     "intercept_lin_fit_2_100",
#     "discharge_capacity_2",
#     "mean_charge_time_2_6",
#     "minimum_IR_2_100",
#     "diff_IR_100_2",
# ]

In [19]:
cell_dfs = []
for cell_k, cell_v in preprocessed_pkl.items():
    print('processing', cell_k)
    # init arrays for values
    total_cycles = len(cell_v['cycles'])
    num_windows = int( (total_cycles-window_size) // shift) + 1
    sanity_check = int(total_cycles//shift)
    print('num_windows', num_windows, 'total_cycles', total_cycles)
    assert num_windows == sanity_check

processing b1c0
num_windows 92 total_cycles 1850
processing b1c1
num_windows 107 total_cycles 2158
processing b1c2
num_windows 111 total_cycles 2234
processing b1c3
num_windows 71 total_cycles 1431
processing b1c4
num_windows 85 total_cycles 1706
processing b1c5
num_windows 53 total_cycles 1071
processing b1c6
num_windows 31 total_cycles 634
processing b1c7
num_windows 43 total_cycles 868
processing b1c9
num_windows 52 total_cycles 1052
processing b1c11
num_windows 39 total_cycles 786
processing b1c14
num_windows 43 total_cycles 878
processing b1c15
num_windows 35 total_cycles 717
processing b1c16
num_windows 43 total_cycles 860
processing b1c17
num_windows 42 total_cycles 855
processing b1c18
num_windows 34 total_cycles 689
processing b1c19
num_windows 39 total_cycles 786
processing b1c20
num_windows 26 total_cycles 531
processing b1c21
num_windows 27 total_cycles 556
processing b1c23
num_windows 50 total_cycles 1012
processing b1c24
num_windows 50 total_cycles 1015
processing b1c25
n

In [20]:
# work out details on specific cell
cell_k = 'b1c0'
cell_v = preprocessed_pkl[cell_k]
cell_cycles = cell_v['cycles']
total_cycles = len(cell_cycles)
cell_summary = cell_v['summary']

In [21]:
# slice cycle keys into windows
cycle_keys = list(cell_cycles.keys())
window_cycle_keys = []
for i, w_slice in enumerate(range(0, len(cycle_keys), shift)):
    cycle_keys_slice = cycle_keys[w_slice : w_slice + window_size]
    if len(cycle_keys_slice) % window_size == 0:  # drop remainder
        window_cycle_keys.append(cycle_keys_slice)

In [22]:
num_windows = len(window_cycle_keys)
# sanity check
assert num_windows == int((total_cycles-window_size) // shift) + 1
print('num_windows', num_windows, 'total_cycles', total_cycles)
minimum_dQ_window = np.zeros(num_windows)
variance_dQ_window = np.zeros(num_windows)
slope_lin_fit_window = np.zeros(num_windows)
intercept_lin_fit_window = np.zeros(num_windows)
discharge_capacity_2 = np.zeros(num_windows)
mean_charge_time_2_6 = np.zeros(num_windows)
minimum_IR_window = np.zeros(num_windows)
diff_IR_window = np.zeros(num_windows)
target_remaining = np.zeros(num_windows)
target_current = np.zeros(num_windows)

num_windows 92 total_cycles 1850


In [23]:
for i, window_keys in enumerate(window_cycle_keys):
    key_c2 = window_keys[1]
    key_clast = window_keys[-1]
    c2 = cell_cycles[key_c2]    
    clast = cell_cycles[key_clast]
    # summary keys are 0 indexed
    summary_key_c2 = int(key_c2) - 1
    summary_key_clast = int(key_clast) - 1
    print("{}: [{}, {}], summary_keys [{}, {}]".format(i, key_c2, key_clast, summary_key_c2, summary_key_clast))

    # 1. delta_Q_100_10(V) -> delta_Q_last_2(V)
    dQ_window = clast['Qdlin'] - c2['Qdlin']
    minimum_dQ_window[i] = np.log(np.abs(np.min(dQ_window)))
    variance_dQ_window[i] = np.log(np.var(dQ_window))
    
    # 2. Discharge capacity fade curve features
    # Compute linear fit for cycles 2 to last:
    # discharge cappacities; q.shape = (window_size-1, 1); 
    q = cell_summary['QD'][summary_key_c2:summary_key_clast].reshape(-1, 1)     
    # Cylce index from 2 to last; X.shape = (window_size-1, 1)
    X = np.array(window_keys)[2:].reshape(-1, 1) 

    linear_regressor_window = LinearRegression()
    linear_regressor_window.fit(X, q)
    slope_lin_fit_window[i] = linear_regressor_window.coef_[0]
    intercept_lin_fit_window[i] = linear_regressor_window.intercept_
    discharge_capacity_2[i] = q[0][0]
        
    # 3. Other features
    summary_key_c6 = int(window_keys[5]) - 1
    mean_charge_time_2_6[i] = np.mean(cell_summary['Discharge_time'][summary_key_c2:summary_key_c6])
    minimum_IR_window[i] = np.min(cell_summary['IR'][summary_key_c2:summary_key_clast])
    diff_IR_window[i] = cell_summary['IR'][summary_key_clast] - cell_summary['IR'][summary_key_c2]

    # 4. Targets
    target_remaining[i] = cell_summary['Remaining_cycles'][summary_key_clast]
    target_current[i] = int(key_clast)

0: [2, 20], summary_keys [1, 19]
1: [22, 40], summary_keys [21, 39]
2: [42, 60], summary_keys [41, 59]
3: [62, 80], summary_keys [61, 79]
4: [82, 100], summary_keys [81, 99]
5: [102, 120], summary_keys [101, 119]
6: [122, 140], summary_keys [121, 139]
7: [142, 160], summary_keys [141, 159]
8: [162, 180], summary_keys [161, 179]
9: [182, 200], summary_keys [181, 199]
10: [202, 220], summary_keys [201, 219]
11: [222, 240], summary_keys [221, 239]
12: [242, 260], summary_keys [241, 259]
13: [262, 280], summary_keys [261, 279]
14: [282, 300], summary_keys [281, 299]
15: [302, 320], summary_keys [301, 319]
16: [322, 340], summary_keys [321, 339]
17: [342, 360], summary_keys [341, 359]
18: [362, 380], summary_keys [361, 379]
19: [382, 400], summary_keys [381, 399]
20: [402, 420], summary_keys [401, 419]
21: [422, 440], summary_keys [421, 439]
22: [442, 460], summary_keys [441, 459]
23: [462, 480], summary_keys [461, 479]
24: [482, 500], summary_keys [481, 499]
25: [502, 520], summary_keys [5

C:\Users\PC-1\AppData\Local\Programs\Python\Python38\Lib\site-packages\sklearn\utils\validation.py:747: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  X = check_array(X, accept_sparse=accept_sparse,
C:\Users\PC-1\AppData\Local\Programs\Python\Python38\Lib\site-packages\sklearn\utils\validation.py:747: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  X = check_array(X, accept_sparse=accept_sparse,
C:\Users\PC-1\AppData\Local\Programs\Python\Python38\Lib\site-packages\sklearn\utils\validation.py:747: FutureWarning: Beginni

In [24]:
cell_summary.keys()

dict_keys(['IR', 'QD', 'Remaining_cycles', 'Discharge_time'])

In [25]:
# assemble cell-level df
cell_df = pd.DataFrame({
    "cell_key": np.array(cell_k),
    "minimum_dQ_100_10": minimum_dQ_window,
    "variance_dQ_window": variance_dQ_window,
    "slope_lin_fit_window": slope_lin_fit_window,
    "intercept_lin_fit_window": intercept_lin_fit_window,
    "discharge_capacity_2": discharge_capacity_2,
    "mean_charge_time_2_6": mean_charge_time_2_6,
    "minimum_IR_window": minimum_IR_window,
    "diff_IR_window": diff_IR_window,
    "target_remaining": np.array(target_remaining),
    "target_current": np.array(target_current)
})

In [26]:
cell_df.head()

,cell_key,minimum_dQ_100_10,variance_dQ_window,slope_lin_fit_window,intercept_lin_fit_window,discharge_capacity_2,mean_charge_time_2_6,minimum_IR_window,diff_IR_window,target_remaining,target_current
0,b1c0,-6.183421,-11.397280,0.000651,1.092588,1.071900,14.410800,0.016565,-0.000145,1832.0,20.0
1,b1c0,-4.721578,-12.903884,0.000024,1.075805,1.076636,14.468635,0.016565,-0.000005,1812.0,40.0
2,b1c0,-5.438847,-11.189355,-0.000023,1.077595,1.076675,14.468870,0.016593,-0.000064,1792.0,60.0
3,b1c0,-3.746048,-10.472811,-0.000043,1.079416,1.077082,14.466924,0.016444,0.000238,1772.0,80.0
4,b1c0,-4.617751,-12.906133,-0.000020,1.077721,1.076188,14.452545,0.016624,-0.000023,1752.0,100.0


In [27]:
cell_dfs = []

for cell_k, cell_v in preprocessed_pkl.items():
    print('processing', cell_k)
    cell_cycles = cell_v['cycles']
    cell_summary = cell_v['summary']
    total_cycles = len(cell_cycles)

    # slice cycle keys into windows
    cycle_keys = list(cell_cycles.keys())
    window_cycle_keys = []
    for i, w_slice in enumerate(range(0, len(cycle_keys), shift)):
        cycle_keys_slice = cycle_keys[w_slice : w_slice + window_size]
        if len(cycle_keys_slice) % window_size == 0:  # drop remainder
            window_cycle_keys.append(cycle_keys_slice)

    # init value arrays
    num_windows = len(window_cycle_keys)
    assert num_windows == int((total_cycles-window_size) // shift) + 1
    print('num_windows', num_windows, 'total_cycles', total_cycles)
    minimum_dQ_window = np.zeros(num_windows)
    variance_dQ_window = np.zeros(num_windows)
    slope_lin_fit_window = np.zeros(num_windows)
    intercept_lin_fit_window = np.zeros(num_windows)
    discharge_capacity_2 = np.zeros(num_windows)
    mean_discharge_time = np.zeros(num_windows)
    minimum_IR_window = np.zeros(num_windows)
    diff_IR_window = np.zeros(num_windows)
    target_remaining = np.zeros(num_windows)
    target_current = np.zeros(num_windows)

    # build cell-level df
    for i, window_keys in enumerate(window_cycle_keys):
        key_c1 = window_keys[0]
        key_clast = window_keys[-1]
        # summary keys may not line up with cycle keys, since cycles could be cleaned up
        summary_key_c1 = np.where(np.array(cycle_keys)==key_c1)[0][0]
        summary_key_clast = np.where(np.array(cycle_keys)==key_clast)[0][0]
        print("{}: [{}, {}], summary_keys: [{}, {}]".format(i, key_c1, key_clast, summary_key_c1, summary_key_clast))

        # 1. delta_Q_100_10(V) -> delta_Q_window(V)
        dQ_window = cell_cycles[key_clast]['Qdlin'] - cell_cycles[key_c1]['Qdlin']
        minimum_dQ_window[i] = np.log(np.abs(np.min(dQ_window)))
        variance_dQ_window[i] = np.log(np.var(dQ_window))

        # 2. Discharge capacity fade curve features
        # Compute linear fit for cycles 2 to last:
        # discharge cappacities; q.shape = (window_size, 1); 
        q = cell_summary['QD'][summary_key_c1:summary_key_clast+1].reshape(-1, 1).astype(np.float64) 
        # Cylce index from 2 to last; X.shape = (window_size, 1)
        X = np.arange(window_size).reshape(-1, 1).astype(np.int32) 

        linear_regressor_window = LinearRegression()
        linear_regressor_window.fit(X, q)
        slope_lin_fit_window[i] = linear_regressor_window.coef_[0]
        intercept_lin_fit_window[i] = linear_regressor_window.intercept_
        discharge_capacity_2[i] = q[0][0]

        # 3. Other features
        mean_discharge_time[i] = np.mean(cell_summary['Discharge_time'][summary_key_c1:summary_key_clast+1])
        minimum_IR_window[i] = np.min(cell_summary['IR'][summary_key_c1:summary_key_clast+1])
        diff_IR_window[i] = cell_summary['IR'][summary_key_clast] - cell_summary['IR'][summary_key_c1]
        
        # 4. Targets
        target_remaining[i] = cell_summary['Remaining_cycles'][summary_key_clast]
        target_current[i] = int(key_clast)

    # assemble cell-level df
    cell_dfs.append(
        pd.DataFrame({
            "cell_key": np.array(cell_k, dtype=str),
            "minimum_dQ_100_10": minimum_dQ_window,
            "variance_dQ_window": variance_dQ_window,
            "slope_lin_fit_window": slope_lin_fit_window,
            "intercept_lin_fit_window": intercept_lin_fit_window,
            "discharge_capacity_2": discharge_capacity_2,
            "mean_discharge_time": mean_discharge_time,
            "minimum_IR_window": minimum_IR_window,
            "diff_IR_window": diff_IR_window,
            "target_remaining": np.array(target_remaining, dtype=np.int32),
            "target_current": np.array(target_current, dtype=np.int32)
        }))
    
df = pd.concat(cell_dfs)

processing b1c0
num_windows 92 total_cycles 1850
0: [1, 20], summary_keys: [0, 19]
1: [21, 40], summary_keys: [20, 39]
2: [41, 60], summary_keys: [40, 59]
3: [61, 80], summary_keys: [60, 79]
4: [81, 100], summary_keys: [80, 99]
5: [101, 120], summary_keys: [100, 119]
6: [121, 140], summary_keys: [120, 139]
7: [141, 160], summary_keys: [140, 159]
8: [161, 180], summary_keys: [160, 179]
9: [181, 200], summary_keys: [180, 199]
10: [201, 220], summary_keys: [200, 219]
11: [221, 240], summary_keys: [220, 239]
12: [241, 260], summary_keys: [240, 259]
13: [261, 280], summary_keys: [260, 279]
14: [281, 300], summary_keys: [280, 299]
15: [301, 320], summary_keys: [300, 319]
16: [321, 340], summary_keys: [320, 339]
17: [341, 360], summary_keys: [340, 359]
18: [361, 380], summary_keys: [360, 379]
19: [381, 400], summary_keys: [380, 399]
20: [401, 420], summary_keys: [400, 419]
21: [421, 440], summary_keys: [420, 439]
22: [441, 460], summary_keys: [440, 459]
23: [461, 480], summary_keys: [460, 479

In [28]:
df.head(100)

,cell_key,minimum_dQ_100_10,variance_dQ_window,slope_lin_fit_window,intercept_lin_fit_window,discharge_capacity_2,mean_discharge_time,minimum_IR_window,diff_IR_window,target_remaining,target_current
0,b1c0,-6.228550,-11.167578,0.000558,1.092129,1.070689,14.391515,0.016565,-0.000163,1832,20
1,b1c0,-4.367995,-12.612905,0.000018,1.076392,1.076654,14.467388,0.016560,-0.000002,1812,40
2,b1c0,-6.163657,-11.004845,-0.000013,1.076579,1.076660,14.467032,0.016539,-0.000035,1792,60
3,b1c0,-3.758712,-10.568806,-0.000043,1.076795,1.077056,14.460795,0.016444,0.000209,1772,80
4,b1c0,-4.470127,-12.248222,-0.000020,1.076095,1.076199,14.448800,0.016624,-0.000022,1752,100
...,...,...,...,...,...,...,...,...,...,...,...
3,b1c1,-4.578280,-12.550202,-0.000033,1.081732,1.081541,14.539522,0.016764,0.000036,2080,80
4,b1c1,-4.770284,-12.740123,-0.000016,1.081010,1.081047,14.523506,0.016928,0.000050,2060,100
5,b1c1,-4.818000,-12.437827,-0.000029,1.080698,1.080636,14.509984,0.016941,0.000025,2040,120
6,b1c1,-4.503605,-12.436978,-0.000033,1.080326,1.080321,14.497227,0.016952,0.000028,2020,140


In [29]:
df[df.cell_key=="b2c14"]

,cell_key,minimum_dQ_100_10,variance_dQ_window,slope_lin_fit_window,intercept_lin_fit_window,discharge_capacity_2,mean_discharge_time,minimum_IR_window,diff_IR_window,target_remaining,target_current
0,b2c14,-4.717688,-11.542532,0.000171,1.079932,1.078068,14.516225,0.016895,-2.279750e-04,465,20
1,b2c14,-4.084237,-11.283040,0.000021,1.082482,1.082609,14.514638,0.016896,8.932900e-05,445,40
2,b2c14,-3.870669,-10.590868,-0.000019,1.082830,1.082851,14.486833,0.016951,3.758100e-05,425,60
3,b2c14,-3.830207,-10.573997,-0.000036,1.082499,1.082546,14.454588,0.016974,5.786000e-06,405,80
4,b2c14,-4.038535,-10.843425,-0.000031,1.081543,1.081668,14.405229,0.016960,-3.724500e-05,385,100
5,b2c14,-3.930737,-10.748697,-0.000069,1.080780,1.080710,14.336249,0.016940,1.271500e-05,365,120
6,b2c14,-3.998870,-11.141231,-0.000077,1.079281,1.079249,14.251762,0.016921,2.009400e-05,345,140
7,b2c14,-4.033829,-10.973223,-0.000098,1.077643,1.077670,14.153636,0.016984,3.337700e-05,325,160
8,b2c14,-3.977480,-11.033810,-0.000125,1.075650,1.075636,14.047388,0.016984,-3.645900e-05,305,180
9,b2c14,-4.012309,-11.102448,-0.000130,1.073209,1.073175,13.941005,0.017011,6.062700e-05,285,200


In [30]:
df.cell_key.value_counts()

b1c2     111
b1c1     107
b1c0      92
b3c7      91
b3c45     89
        ... 
b2c2      21
b2c46     21
b2c3      16
b2c0      15
b2c1       7
Name: cell_key, Length: 124, dtype: int64

In [31]:
df.shape

(4881, 11)

In [32]:
cycle_keys

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '13